In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("test") \
        .getOrCreate()

23/03/07 10:55:19 WARN Utils: Your hostname, warcry98 resolves to a loopback address: 127.0.1.1; using 192.168.100.64 instead (on interface wlp3s0)
23/03/07 10:55:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/07 10:55:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.parquet('Data/fhvhv/2021/06/')

In [4]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [5]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02875|2021-06-16 17:30:56|2021-06-16 18:08:31|         141|          68|      N|                B02875|
|              B02871|2021-06-25 03:46:52|2021-06-25 04:09:45|          68|          68|      N|                B02871|
|              B02510|2021-06-01 08:50:09|2021-06-01 09:17:40|         216|         134|      N|                  null|
|              B02884|2021-06-15 19:05:01|2021-06-15 19:29:42|         163|         114|      N|                B02884|
|              B02765|2021-06-17 11:53:53|2021-06-17 12:11:28|          77|         155|      N|                B02765|
|              B02884|2021-06-18 17:59:0

In [6]:
from pyspark.sql import functions as F

In [28]:
df_date = df \
  .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
  .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
  .select('dispatching_base_num', 'pickup_date', 'dropoff_date', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID')

In [29]:
df_date.show()

+--------------------+-----------+------------+-------------------+-------------------+------------+------------+
|dispatching_base_num|pickup_date|dropoff_date|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+--------------------+-----------+------------+-------------------+-------------------+------------+------------+
|              B02875| 2021-06-16|  2021-06-16|2021-06-16 17:30:56|2021-06-16 18:08:31|         141|          68|
|              B02871| 2021-06-25|  2021-06-25|2021-06-25 03:46:52|2021-06-25 04:09:45|          68|          68|
|              B02510| 2021-06-01|  2021-06-01|2021-06-01 08:50:09|2021-06-01 09:17:40|         216|         134|
|              B02884| 2021-06-15|  2021-06-15|2021-06-15 19:05:01|2021-06-15 19:29:42|         163|         114|
|              B02765| 2021-06-17|  2021-06-17|2021-06-17 11:53:53|2021-06-17 12:11:28|          77|         155|
|              B02884| 2021-06-18|  2021-06-18|2021-06-18 17:59:09|2021-06-18 18:01:50| 

In [57]:
df_time_diff = df_date \
  .withColumn("DiffInSeconds", (F.unix_timestamp('dropoff_datetime') - F.unix_timestamp('pickup_datetime'))) \
  .withColumn("DiffInHours", ((F.col("DiffInSeconds")/3600)))

In [58]:
df_time_diff.show()

+--------------------+-----------+------------+-------------------+-------------------+------------+------------+-------------+-------------------+
|dispatching_base_num|pickup_date|dropoff_date|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|DiffInSeconds|        DiffInHours|
+--------------------+-----------+------------+-------------------+-------------------+------------+------------+-------------+-------------------+
|              B02875| 2021-06-16|  2021-06-16|2021-06-16 17:30:56|2021-06-16 18:08:31|         141|          68|         2255| 0.6263888888888889|
|              B02871| 2021-06-25|  2021-06-25|2021-06-25 03:46:52|2021-06-25 04:09:45|          68|          68|         1373| 0.3813888888888889|
|              B02510| 2021-06-01|  2021-06-01|2021-06-01 08:50:09|2021-06-01 09:17:40|         216|         134|         1651|0.45861111111111114|
|              B02884| 2021-06-15|  2021-06-15|2021-06-15 19:05:01|2021-06-15 19:29:42|         163|         114

In [59]:
df_max_time = df_time_diff.groupBy("dispatching_base_num", "pickup_date").max("DiffInHours")

In [62]:
df_max_time.sort("max(DiffInHours)", ascending=False).show()

+--------------------+-----------+------------------+
|dispatching_base_num|pickup_date|  max(DiffInHours)|
+--------------------+-----------+------------------+
|              B02872| 2021-06-25|  66.8788888888889|
|              B02765| 2021-06-22|25.549722222222222|
|              B02879| 2021-06-27|19.980833333333333|
|              B02800| 2021-06-26|18.197222222222223|
|              B02682| 2021-06-23|16.466944444444444|
|              B02869| 2021-06-23|14.268888888888888|
|              B02877| 2021-06-24|13.909722222222221|
|              B02765| 2021-06-04|             11.67|
|              B02617| 2021-06-27|11.365833333333333|
|              B02880| 2021-06-20|10.984444444444444|
|              B02866| 2021-06-01|           10.2675|
|              B02882| 2021-06-28| 9.966388888888888|
|              B02510| 2021-06-01| 9.966388888888888|
|              B02510| 2021-06-27| 9.637777777777778|
|              B02510| 2021-06-18| 9.624444444444444|
|              B02510| 2021-